In [1]:
%matplotlib inline
%load_ext autoreload
# %load_ext toc2
%autoreload 2

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pickle
import itertools
from datetime import datetime


import glob
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import IPython

from pymo.parsers import BVHParser
from pymo.preprocessing import *
from pymo.viz_tools import *
from pymo.writers import *

In [2]:
import csv
import numpy as np
import math
motion_data_temp = np.zeros((3476,78),dtype=float)
motion_data = np.zeros((3476,72),dtype=float)
with open('data/Walking_1_1.txt') as csvfile:
    reader = csv.reader(csvfile,delimiter=",",quotechar='|')
    i=0
    for row in reader:
        motion_data_temp[i,:] = np.array(row)
        i=i+1
print(i)
number_of_frames = i
skip =0

#H36M_Mapping = [0,1,2,3,4,5,6,7,8,9,10,11,12]
#H36M_Mapping = [c for c in range(0,26)]
#print(H36M_Mapping)
H36M_Mapping = [0,1,10,13,14,15,16,17,18,19,20,21,22,23,24,25,6,7,8,9,2,3,4,5]

for frame in range(number_of_frames):
    for i in range(26):

        if i>23:
            break;
        if(i == 0) : 
            if(frame ==0):
                motion_data[0,0] = -6.5115
                motion_data[0,1] = 35.5548
                motion_data[0,2] = -163.0160
            else:
#                 motion_data[frame,((i-skip)*3)+0] = motion_data_temp[frame,(i-skip)*3+0]/100
#                 motion_data[frame,((i-skip)*3)+1] = motion_data_temp[frame,(i-skip)*3+1]/100
#                 motion_data[frame,((i-skip)*3)+2] = motion_data_temp[frame,(i-skip)*3+2]/100
                temp1 = motion_data_temp[frame,H36M_Mapping[i]*3+0]/10000
                temp2 = motion_data_temp[frame,H36M_Mapping[i]*3+1]/10000
                temp3 = motion_data_temp[frame,H36M_Mapping[i]*3+2]/10000
        
                motion_data[frame,((i-skip)*3)+0] = (temp2 + motion_data[frame-1,i*3+0])
                motion_data[frame,((i-skip)*3)+1] = (temp3 + motion_data[frame-1,i*3+1])
                motion_data[frame,((i-skip)*3)+2] = (temp1 + motion_data[frame-1,i*3+2])
        else:
            temp1 = motion_data_temp[frame,H36M_Mapping[i]*3+0]/10000
            temp2 = motion_data_temp[frame,H36M_Mapping[i]*3+1]/10000
            temp3 = motion_data_temp[frame,H36M_Mapping[i]*3+2]/10000
                
            motion_data[frame,((i)*3)+0] = (temp2 + motion_data_temp[frame-1,i*3+0])#*(math.pi)/180
            motion_data[frame,((i)*3)+1] = (temp3 + motion_data_temp[frame-1,i*3+1])#*(math.pi)/180
            motion_data[frame,((i)*3)+2] = (temp1 + motion_data_temp[frame-1,i*3+2])#*(math.pi)/180


from shutil import copyfile
destination_file = 'data/S1_Walking_1_1.bvh'
copyfile('data/BVH_header', destination_file)
with open(destination_file,'a') as f: 
    f.write("\nFrames: {0}".format(number_of_frames))
    f.write("\nFrame Time: 0.008333\n")
with open(destination_file, 'a') as csvfile:
    bvhwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for i in range(number_of_frames):
        bvhwriter.writerow(motion_data[i,:])

IOError: [Errno 2] No such file or directory: 'data/Walking_1_1.txt'

In [3]:
p = BVHParser()
data_all = [p.parse(destination_file)]
# print_skel(data_all)
BVH2Pos = MocapParameterizer('position')
data_pos = BVH2Pos.fit_transform(data_all)
print(data_pos[0].values[0:0])

Empty DataFrame
Columns: [Hips_Xposition, Hips_Yposition, Hips_Zposition, RightUpLeg_Xposition, RightUpLeg_Yposition, RightUpLeg_Zposition, RightLeg_Xposition, RightLeg_Yposition, RightLeg_Zposition, RightFoot_Xposition, RightFoot_Yposition, RightFoot_Zposition, RightToeBase_Xposition, RightToeBase_Yposition, RightToeBase_Zposition, RightToeBase_Nub_Xposition, RightToeBase_Nub_Yposition, RightToeBase_Nub_Zposition, LeftUpLeg_Xposition, LeftUpLeg_Yposition, LeftUpLeg_Zposition, LeftLeg_Xposition, LeftLeg_Yposition, LeftLeg_Zposition, LeftFoot_Xposition, LeftFoot_Yposition, LeftFoot_Zposition, LeftToeBase_Xposition, LeftToeBase_Yposition, LeftToeBase_Zposition, LeftToeBase_Nub_Xposition, LeftToeBase_Nub_Yposition, LeftToeBase_Nub_Zposition, Spine_Xposition, Spine_Yposition, Spine_Zposition, RightShoulder_Xposition, RightShoulder_Yposition, RightShoulder_Zposition, RightArm_Xposition, RightArm_Yposition, RightArm_Zposition, RightForeArm_Xposition, RightForeArm_Yposition, RightForeArm_Zpos

In [4]:
p = BVHParser()
data_all = [p.parse('./data/AV_8Walk_Meredith_HVHA_Rep1.bvh')]
# data_all = [p.parse('./data/motion1.bvh')]
# data_all = [p.parse('./data/h36m_walking.bvh')]
#data_all = [p.parse(destination_file)]

BVH2Pos = MocapParameterizer('position')
data_pos = BVH2Pos.fit_transform(data_all)
# print_skel(data_pos)
nb_play_mocap(data_pos[0], 'pos', scale=2, camera_z=800, frame_time=0.1, base_url='mocapplayer/playBuffer.html')